# Map the competitive routes

In [ ]:
import branca
import geopandas as gpd
import intake
import os
import numpy as np
import pandas as pd

import utils
from shared_utils import map_utils, geography_utils
from shared_utils import calitp_color_palette as cp

catalog = intake.open_catalog("./*.yml")

In [ ]:
gdf = catalog.gmaps_results.read()

# Need a numeric identifier, or branca.colormap will error
gdf = gdf.assign(
    # Try to sort/group slightly differently or else colors for same operator appear too similar
    identifier2 = gdf.apply(lambda x: np.random.randint(0, 1619), axis=1),
)

In [ ]:
FIG_HEIGHT = 900
FIG_WIDTH = 700

'''
color_palette = (cp.CALITP_CATEGORY_BRIGHT_COLORS + cp.CALITP_CATEGORY_BOLD_COLORS + 
                 cp.CALITP_DIVERGING_COLORS +  cp.CALITP_SEQUENTIAL_COLORS
                )

random.seed(1234)
random.shuffle(color_palette)
'''
COLORSCALE = branca.colormap.step.Accent_08.scale(
    vmin = gdf.identifier2.min(),
    vmax=gdf.identifier2.max(),
)
#(colors = color_palette)

# Choose a quantitative column to plot, 
# otherwise using categorical/discrete colors will throw error
# TypeError: '<=' not supported between instances of 'str' and 'float'

PLOT_COL = "identifier2"
POPUP_DICT = {
    "calitp_itp_id": "ITP ID",
    "route_id": "Route ID",
    "trip_id": "Trip ID for 20th percentile travel time",
    "service_hours": "Bus Travel Time (hrs)",
    "car_duration_hours": "Car Travel Time (hrs)",
    "shape_id": "shape_id",
}

In [ ]:
m = map_utils.make_folium_choropleth_map(
    # Drop datetime, arrays cols because it's not JSON serializable
    gdf[gdf.competitive==1].drop(columns = 
                                 ["trip_departure", "departure_in_one_year", 
                                  "origin", "destination", "waypoints"]),
    plot_col = PLOT_COL,
    popup_dict = POPUP_DICT,
    tooltip_dict = POPUP_DICT,
    colorscale = COLORSCALE,
    fig_width = FIG_WIDTH,
    fig_height = FIG_HEIGHT,
    zoom=map_utils.REGION_CENTROIDS["CA"]["zoom"]+1,
    centroid=map_utils.REGION_CENTROIDS["CA"]["centroid"],
    title="Competitive Transit Routes",
    legend_name="Legend",
)

m.save(f"{utils.IMG_PATH}competitive_routes.html")

In [ ]:
#m